1. What is the fundamental idea behind Support Vector Machines?

using data points as support vectors to find a street between classes (for classification) or a wide street over data (for regression)

2. What is a support vector?

a support vector represents the data points in and on the edge of the street for support vector machines. The decision boundary is entirelydetermined by the support vectors. Any instance that is not a support vector (i.e., is off the street) has no influence whatsoever; you could remove them, add more instances, or move them around, and as long as they stay off the street they won’t affect the decision boundary.

3. Why is it important to scale the inputs when using SVMs?

else the margin defind is almost orthogonal to the much large valude feature

4. Can an SVM classifier output a confidence score when it classifies an instance?
What about a probability?

doesnt provide probablity as margin is not probablistic dstance based. <span style="color:red;">If you set
probability=True when creating an SVM in Scikit-Learn, then after training it
will calibrate the probabilities using Logistic Regression on the SVM’s scores
(trained by an additional five-fold cross-validation on the training data). This
will add the predict_proba() and predict_log_proba() methods to the SVM</span>

5. Should you use the primal or the dual form of the SVM problem to train a model
on a training set with millions of instances and hundreds of features?

<span style="color:red;">This question applies only to linear SVMs since kernelized SVMs can only use
the dual form. The computational complexity of the primal form of the SVM
problem is proportional to the number of training instances m, while the computational
complexity of the dual form is proportional to a number between m2 and
m3. So if there are millions of instances, you should definitely use the primal
form, because the dual form will be much too slow.</span>

6. Say you’ve trained an SVM classifier with an RBF kernel, but it seems to underfit
the training set. Should you increase or decrease γ (gamma)? What about C?

increase gamma and/or C (regularization)

7. How should you set the QP parameters (H, f, A, and b) to solve the soft margin
linear SVM classifier problem using an off-the-shelf QP solver?

<pre style="color:red;">Let’s call the QP parameters for the hard margin problem H′, f′, A′, and b′ (see
“Quadratic Programming” on page 167). The QP parameters for the soft margin
problem have m additional parameters (np = n + 1 + m) and m additional constraints
(nc = 2m). They can be defined like so:
• H is equal to H′, plus m columns of 0s on the right and m rows of 0s at the
bottom: H =
H′ 0 ⋯
0 0
⋮ ⋱
• f is equal to f′ with m additional elements, all equal to the value of the hyperparameter
C.
• b is equal to b′ with m additional elements, all equal to 0.
• A is equal to A′, with an extra m × m identity matrix Im appended to the right,
–*I*m just below it, and the rest filled with 0s: A =
A′ Im
0 −Im
</pre>

8. Train a LinearSVC on a linearly separable dataset. Then train an SVC and a
SGDClassifier on the same dataset. See if you can get them to produce roughly
the same model.

In [1]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

setosa_or_versicolor_cond = (y == 0) | (y == 1)
X = X[setosa_or_versicolor_cond]
y = y[setosa_or_versicolor_cond]

In [5]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier

lin_svc = LinearSVC(loss="hinge", C=10, random_state=2)
svc = SVC(kernel="linear", C=10)
sgdc = SGDClassifier(loss="hinge", learning_rate='constant', eta0=0.2, random_state=2)

In [6]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_final = ss.fit_transform(X)

In [8]:
lin_svc.fit(x_final, y)
svc.fit(x_final, y)
sgdc.fit(x_final, y)

print("model intercept coeff")
print("LinearSVC", lin_svc.intercept_, lin_svc.coef_)
print("SVC", svc.intercept_, svc.coef_)
print("SGDC", sgdc.intercept_, sgdc.coef_)

model intercept coeff
LinearSVC [0.28474837] [[1.05365141 1.09903944]]
SVC [0.31896852] [[1.1203284  1.02625193]]
SGDC [0.4] [[1.2965822  1.16781349]]


9. Train an SVM classifier on the MNIST dataset. Since SVM classifiers are binary
classifiers, you will need to use one-versus-the-rest to classify all 10 digits. You
may want to tune the hyperparameters using small validation sets to speed up the
process. What accuracy can you reach?

In [21]:
from sklearn import datasets 

mnist = datasets.load_digits()
x = mnist["data"]
y = mnist["target"]

In [20]:
x.shape

(1797, 64)

In [26]:
import numpy as np
full = np.hstack((x, y.reshape(-1, 1)))

In [27]:
np.random.shuffle(full)

x_train = full[:1500, :-1]
x_test = full[1500:, :-1]
y_train = full[:1500, -1]
y_test = full[1500:, -1]

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(1500, 64)
(297, 64)
(1500,)
(297,)


In [28]:
lin_svc1 = LinearSVC(random_state=24)
lin_svc1.fit(x_train, y_train)

c:\users\abhishek\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=24, tol=0.0001,
          verbose=0)

In [30]:
from sklearn.metrics import accuracy_score
y_pred1 = lin_svc1.predict(x_test)

accuracy_score(y_test, y_pred1)

0.9393939393939394

In [31]:
ss1 = StandardScaler()
x_train_scaled1 = ss1.fit_transform(x_train)
x_test_scaled1 = ss1.transform(x_test)

In [32]:
lin_svc2 = LinearSVC(random_state=24)
lin_svc2.fit(x_train_scaled1, y_train)

c:\users\abhishek\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=24, tol=0.0001,
          verbose=0)

In [33]:
y_pred2 = lin_svc2.predict(x_test_scaled1)
accuracy_score(y_test, y_pred2)

0.9393939393939394

In [39]:
svc1 = SVC(gamma="scale")
svc1.fit(x_train_scaled1, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [40]:
y_pred3 = svc1.predict(x_test_scaled1)
accuracy_score(y_test, y_pred3)

0.9797979797979798

In [41]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_range = {"gamma": reciprocal(0.0001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(svc1, param_range, n_iter=20, verbose=2, cv=3)
rnd_search_cv.fit(x_train_scaled1, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=10.860920112874624, gamma=0.003549859458034439 ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . C=10.860920112874624, gamma=0.003549859458034439, total=   0.3s
[CV] C=10.860920112874624, gamma=0.003549859458034439 ................
[CV] . C=10.860920112874624, gamma=0.003549859458034439, total=   0.1s
[CV] C=10.860920112874624, gamma=0.003549859458034439 ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


[CV] . C=10.860920112874624, gamma=0.003549859458034439, total=   0.2s
[CV] C=4.635392581979675, gamma=0.03411769006296352 ..................
[CV] ... C=4.635392581979675, gamma=0.03411769006296352, total=   0.3s
[CV] C=4.635392581979675, gamma=0.03411769006296352 ..................
[CV] ... C=4.635392581979675, gamma=0.03411769006296352, total=   0.3s
[CV] C=4.635392581979675, gamma=0.03411769006296352 ..................
[CV] ... C=4.635392581979675, gamma=0.03411769006296352, total=   0.3s
[CV] C=8.014168864866445, gamma=0.000511732565913184 .................
[CV] .. C=8.014168864866445, gamma=0.000511732565913184, total=   0.2s
[CV] C=8.014168864866445, gamma=0.000511732565913184 .................
[CV] .. C=8.014168864866445, gamma=0.000511732565913184, total=   0.1s
[CV] C=8.014168864866445, gamma=0.000511732565913184 .................
[CV] .. C=8.014168864866445, gamma=0.000511732565913184, total=   0.1s
[CV] C=1.4588997207781187, gamma=0.00346627827429015 .................
[CV] .

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   13.3s finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='scale', kernel='rbf',
                                 max_iter=-1, probability=False,
                                 random_state=None, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn', n_iter=20, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228FEDE5278>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228FCC8EC88>},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [42]:
rnd_search_cv.best_estimator_

SVC(C=10.860920112874624, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.003549859458034439,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [43]:
rnd_search_cv.best_params_

{'C': 10.860920112874624, 'gamma': 0.003549859458034439}

In [44]:

rnd_search_cv.best_score_

0.984

In [46]:
rnd_search_cv.best_estimator_.fit(x_train_scaled1, y_train)

SVC(C=10.860920112874624, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.003549859458034439,
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [48]:
y_pred4 = rnd_search_cv.best_estimator_.predict(x_test_scaled1)
accuracy_score(y_test, y_pred4)

0.9831649831649831

10. Train an SVM regressor on the California housing dataset.

In [51]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
x = housing["data"]
y = housing["target"]

In [54]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [56]:
ss_housing = StandardScaler()
X_train_scaled = ss_housing.fit_transform(X_train)
X_test_scaled = ss_housing.transform(X_test)

In [57]:
from sklearn.svm import LinearSVR

lin_svr = LinearSVR(random_state=24)
lin_svr.fit(X_train_scaled, y_train)

c:\users\abhishek\appdata\local\programs\python\python36\lib\site-packages\sklearn\svm\base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=24, tol=0.0001, verbose=0)

In [58]:
from sklearn.metrics import mean_squared_error

y_pred = lin_svr.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
mse

0.8794326114193507

In [59]:
np.sqrt(mse)

0.9377806840724279

In [60]:
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
rnd_search_cv = RandomizedSearchCV(SVR(), param_distributions, n_iter=10, verbose=2, cv=3, random_state=42)
rnd_search_cv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=  16.9s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.8s remaining:    0.0s


[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=  13.8s
[CV] C=4.745401188473625, gamma=0.07969454818643928 ..................
[CV] ... C=4.745401188473625, gamma=0.07969454818643928, total=  15.0s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=  12.2s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=  12.5s
[CV] C=8.31993941811405, gamma=0.015751320499779724 ..................
[CV] ... C=8.31993941811405, gamma=0.015751320499779724, total=  14.2s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=  11.7s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .. C=2.560186404424365, gamma=0.002051110418843397, total=  10.5s
[CV] C=2.560186404424365, gamma=0.002051110418843397 .................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  6.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='auto_deprecated',
                                 kernel='rbf', max_iter=-1, shrinking=True,
                                 tol=0.001, verbose=False),
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228FCA15F60>,
                                        'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000228FEDD87B8>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring=None, verbose=2)

In [61]:
rnd_search_cv.best_estimator_

SVR(C=4.745401188473625, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma=0.07969454818643928, kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [62]:
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
mse = mean_squared_error(y_train, y_pred)
np.sqrt(mse)

0.5746589015117773

In [63]:
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
np.sqrt(mse)

0.5837289635647664